<a href="https://colab.research.google.com/github/vlozg/speech_hmm/blob/main/Test_DiagHMM_006.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LƯU Ý: NOTEBOOK NÀY CHỈ DÙNG ĐỂ SHOW KẾT QUẢ CHẠY, KHÔNG NÊN CHẠY LẠI NOTEBOOK NÀY 
(VÌ TRONG NÀY CÓ CODE LƯU LẠI PRETRAINED MODEL LÊN DRIVE SẼ BỊ XÓA)

# Speech to text with HMM

- **Bài toán**: Chuyển giọng nói thành văn bản
    - **Input**: Đoạn ghi âm chứa nội dung là các số từ 0 đến 9
    - **Output**: Phân lớp của đoạn ghi âm

# Các biến thiết lập cho thử nghiệm

In [ ]:
n_mfcc_ceptrum = 12
n_delta_features = 0
fsdd_split = 0.4
wolfram_split = 0
experiment_id = '006'

In [ ]:
n_mfcc_features = n_mfcc_ceptrum * (1+n_delta_features)
n_mfcc_features

12

# Import và cài đặt thư viện

In [ ]:
# cài lib. note: cài xong phải restart runtime
!pip install pydub
!pip install pomegranate

In [ ]:
# Xác thực google để upload/download qua google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Quản lý file, folder
import os
from shutil import copyfile, rmtree
import random

# Xử lý audio
import librosa
import librosa.display
from scipy.io import wavfile

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pomegranate # Thư viện cho mô hình xác suất
from pomegranate import *

# Tải dữ liệu và lấy xác thực Google

Dữ liệu dùng để huấn luyện và đánh giá, còn xác thực google thì dùng để upload/download mô hình trên drive.

In [ ]:
%%capture
# download wolfram
if not os.path.isfile('./dataset_1_wolfram.zip'):
  !gdown --id 115tIAitBNeJC0DwrP-ZyJ6RS3TyWN0qD
  !unzip -o dataset_1_wolfram.zip

# dowload FSDD
if not os.path.isfile('./dataset_2_FSDD.zip'):
  !gdown --id 1Ua9zlPBc0Fv4xGHSQTb7eIvUh_dqFI6P
  !unzip -o dataset_2_FSDD.zip

# download self recorded audio
!gdown --id 1lH_k1AYMVlJvodtZdD7OK2zkdPXxlW9i

In [ ]:
# Lấy xác thực google để upload/download file
auth.authenticate_user()
gauth =  GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Hàm xử lý âm thanh

In [ ]:
def scaleAddNoise(wave):
  return ((wave - wave.mean()) / wave.std()) + 0.2*np.random.normal(size=wave.shape)

def read_process_audio(full_audio_path):
  # Bắt buộc âm thanh đọc vào phải cùng sample rate
  sample_rate, wave =  wavfile.read(full_audio_path)
  if sample_rate != 16000:
    wave, sample_rate =  librosa.load(full_audio_path, sr=16000) # Hàm đọc của librosa quá chậm, scipy nhanh gấp 5-10 lần
  else:
    wave=wave/32768 # Chuẩn hóa về số thực
  wave = np.pad(wave, (2000,2000), 'constant', constant_values=(0.0,0.0))
  wave = scaleAddNoise(wave)
  return wave, sample_rate

def extract_mfcc(wave, sample_rate):
  mfccs = librosa.feature.mfcc(y=wave, sr=sample_rate, n_mfcc=n_mfcc_ceptrum+2, n_mels=40, lifter=40)[2:,:].T/800
  return mfccs

def mfcc_delta_features(mfcc, order):
  if order==0:
    return mfcc
  dmfcc = librosa.feature.delta(mfcc, order=order)
  return dmfcc

def full_mfcc_from_file(full_audio_path):
  wave, sr = read_process_audio(full_audio_path)
  mfccs = extract_mfcc(wave, sr)
  full_mfccs = mfccs
  if n_delta_features >= 1:
    mfccs_d1 = mfcc_delta_features(mfccs, 1)
    full_mfccs = np.hstack([full_mfccs,mfccs_d1])
  if n_delta_features >= 2:
    mfccs_d2 = mfcc_delta_features(mfccs, 2)
    full_mfccs = np.hstack([full_mfccs,mfccs_d2])
  return full_mfccs

# Hàm bổ trợ

In [ ]:
def buildDataSet(dir):
    # Filter out the wav audio files under the dir
    fileList = [f for f in os.listdir(dir) if os.path.splitext(f)[1] == '.wav']
    dataset = {}
    for fileName in fileList:
        tmp = fileName.split('.')[0]
        label = tmp.split('_')[1]

        # label = filename.split('_')[0]
        feature = full_mfcc_from_file(dir+fileName)
        if label not in dataset.keys():
            dataset[label] = []
            dataset[label].append(feature)
        else:
            exist_feature = dataset[label]
            exist_feature.append(feature)
            dataset[label] = exist_feature
    return dataset

def makeTrainTestDir(mainFolder, filenames, rate):
    paths = [f'{mainFolder}',
             f'{mainFolder}/train_audio',
             f'{mainFolder}/test_audio']

    for path in paths:
        try:
            os.mkdir(path)
        except:
            rmtree(path)
            os.mkdir(path)

    random.seed(1)
    random.shuffle(filenames)
    splitPoint = int(len(filenames)*rate)
    trainFilenames = filenames[:splitPoint]
    testFilenames = filenames[splitPoint:]

    for filename in trainFilenames:
        copyfile(filename[0], f"{paths[1]}/{filename[1]}")

    for filename in testFilenames:
        copyfile(filename[0], f"{paths[2]}/{filename[1]}")

# Setup thư mục chứa data

In [ ]:
def formatFilenameFSDD(dir):
    filenames = []
    count = 0
    for filename in os.listdir(dir):
        tmp = str(count) + '_' + filename.split('_')[0] + '.wav'
        filenames.append((f"{dir}/{filename}", tmp))
        count += 1
    return filenames

filenames = formatFilenameFSDD('./dataset_2_FSDD')
makeTrainTestDir('fsdd', filenames, fsdd_split)

In [ ]:
def formatFilenameWolfram(dir):
    filenames = []
    count = 0
    folders = os.listdir(dir)
    for folder in folders:
        for filename in os.listdir(f'{dir}/{folder}'):
            tmp = str(count) + '_' + folder.split('_')[0] + '.wav'
            filenames.append((f"{dir}/{folder}/{filename}", tmp))
            count += 1

    return filenames

filenames = formatFilenameWolfram('./dataset_1_wolfram')
makeTrainTestDir('wolfram', filenames, wolfram_split)

# Huấn luyện mô hình (có thể bỏ qua vì mô hình đã save trên drive)

## Hàm train mô hình

In [ ]:
def Generate_DiagGMM(full_fset, n_features, n_states=5, n_cmps=3):
  means = np.mean(full_fset, axis=0)
  stds = np.std(full_fset, axis=0)
  # initial values for all gaussian components
  np.random.seed(None)
  dist_init = np.random.random((n_states, n_cmps, n_features, 2))
  dist_init[..., 0] -= 0.5  # center means to 0.0
  #for feat_i in range(n_features):
      # random init mean in range [-std, std)
      #dist_init[..., feat_i, 0] *= 2 * stds[feat_i]
      #dist_init[..., feat_i, 0] += means[feat_i]
      # random init std in range 1std/n_components
      #dist_init[..., feat_i, 1] *= stds[feat_i] / n_cmps

  dists = tuple(
      GeneralMixtureModel(list(
          IndependentComponentsDistribution(tuple(
              NormalDistribution(*dist_init[state_i, cmp_i, feat_i, :])
              for feat_i in range(n_features)
          ))
          for cmp_i in range(n_cmps)
      ))
      if n_cmps > 1 else
      IndependentComponentsDistribution(tuple(
          NormalDistribution(*dist_init[state_i, 0, feat_i, :])
          for feat_i in range(n_features)
      ))
      for state_i in range(n_states)
  )
  return dists

In [ ]:
# Generate progressive HMM model
def left_right_GMMHMM(seed_sample, x_dim, n_states=10, n_modals=9, random=0):
  rng = np.random.RandomState(random)
  if random!=0:
    init_prob = lambda: rng.rand(1)[0]
    
  model = HiddenMarkovModel()
  states = [State( state_model, name=f"H{i}" ) for i, state_model in enumerate(Generate_DiagGMM(seed_sample, x_dim, n_states, n_modals))]
  model.add_states(states)
  model.add_transition(model.start, states[0], 1)
  for i in range(n_states-1):
    model.add_transition(states[i], states[i], 0.5)
    model.add_transition(states[i], states[i+1], 0.5)
  model.add_transition(states[n_states-1], states[n_states-1], 0.5)
  model.add_transition(states[n_states-1], model.end, 0.5)
  model.bake()
  return model

In [ ]:
def train_GMMHMM(dataset, input_dim, n_hidden_state, n_gauss_modal):
    GMMHMM_Models = dict()

    for label in dataset.keys():
        print(f"Training model detect {label}")
        model = left_right_GMMHMM(dataset[label][0], input_dim, n_hidden_state, n_gauss_modal)
        model.fit(dataset[label], verbose=True, multiple_check_input=False)  # get optimal parameters
        GMMHMM_Models[label] = model

    return GMMHMM_Models

## Hàm test mô hình

In [ ]:
# test model
def evaluateModel(testDataSet, model):
    if (len(testDataSet) == 0):
      return
    score_cnt = 0
    total = 0
    for label in testDataSet.keys():
        features = testDataSet[label]
        total += len(features)
        for sample in features:
            scoreList = {}
            for model_label in model.keys():
                score = model[model_label].log_probability(sample)
                scoreList[model_label] = score
            predict = max(scoreList, key=scoreList.get)
            if predict == label:
                score_cnt += 1
        print(f"{label}: ({score_cnt}/{total})")

    print("Final recognition rate is %.2f"%(100.0*score_cnt/total), "%")

## Đọc và tiền xử lý data

In [ ]:
# prepare data for training
master_path = 'fsdd'

trainDir = master_path + '/train_audio/'
trainDataSet_fsdd = buildDataSet(trainDir)
print("Finish prepare the training data")

# prepare data for testing
testDir = master_path + '/test_audio/'
testDataSet_fsdd = buildDataSet(testDir)
print("Finish prepare the test data")

Finish prepare the training data
Finish prepare the test data


In [ ]:
# prepare data for training
master_path = 'wolfram'

trainDir = master_path + '/train_audio/'
trainDataSet_wolfram = buildDataSet(trainDir)
print("Finish prepare the training data")

# prepare data for testing
testDir = master_path + '/test_audio/'
testDataSet_wolfram = buildDataSet(testDir)
print("Finish prepare the test data")

Finish prepare the training data
Finish prepare the test data


In [ ]:
trainDataSet = trainDataSet_fsdd
if wolfram_split > 0:
  for label in trainDataSet_fsdd.keys():
    trainDataSet[label] = trainDataSet_fsdd[label] + trainDataSet_wolfram[label]

## **Mô hình 1**
- 10 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,10,1)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 104591.77925526214	Time (s): 0.02744
[2] Improvement: 821.9331615241608	Time (s): 0.02421
[3] Improvement: 470.23543668485945	Time (s): 0.024
[4] Improvement: 167.045936951341	Time (s): 0.02722
[5] Improvement: 47.44324765561032	Time (s): 0.02509
[6] Improvement: 49.78564434083091	Time (s): 0.02719
[7] Improvement: 77.55842149543605	Time (s): 0.02429
[8] Improvement: 80.8086900024282	Time (s): 0.02426
[9] Improvement: 46.11472463612881	Time (s): 0.02473
[10] Improvement: 24.3372488629102	Time (s): 0.02449
[11] Improvement: 17.976021411581314	Time (s): 0.02504
[12] Improvement: 19.010631139346515	Time (s): 0.02477
[13] Improvement: 13.189785851769557	Time (s): 0.02871
[14] Improvement: 8.599402887870383	Time (s): 0.0294
[15] Improvement: 9.047416676599823	Time (s): 0.02473
[16] Improvement: 11.5351966672024	Time (s): 0.02437
[17] Improvement: 9.446549165804754	Time (s): 0.02594
[18] Improvement: 8.072902799831354	Time (s): 0.02541
[19] Improvemen

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (81/117)
2: (145/233)
7: (195/327)
6: (250/470)
8: (347/599)
0: (424/721)
9: (502/840)
5: (600/970)
3: (662/1074)
4: (746/1200)
Final recognition rate is 62.17 %
Evaluate on rest of FSDD set
0: (95/178)
3: (175/374)
9: (273/555)
7: (357/761)
4: (476/935)
5: (590/1105)
8: (706/1276)
1: (828/1459)
2: (915/1643)
6: (951/1800)
Final recognition rate is 52.83 %
Evaluate on wolfram test set
4: (329/2372)
8: (676/4724)
7: (1165/7101)
1: (1986/9471)
2: (2102/11844)
3: (2676/14200)
6: (3276/16569)
0: (3916/18945)
5: (4902/21302)
9: (5322/23666)
Final recognition rate is 22.49 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_10_1[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()

## **Mô hình 2**
- 5 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,5,1)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 119583.03223536801	Time (s): 0.01456
[2] Improvement: 512.2381981258368	Time (s): 0.01488
[3] Improvement: 576.9110935415738	Time (s): 0.01518
[4] Improvement: 330.236357237889	Time (s): 0.01529
[5] Improvement: 130.64369255989004	Time (s): 0.01523
[6] Improvement: 83.09051265146991	Time (s): 0.01529
[7] Improvement: 59.70094345785765	Time (s): 0.01483
[8] Improvement: 9.456883365040994	Time (s): 0.02038
[9] Improvement: 1.9744652321314788	Time (s): 0.0152
[10] Improvement: 1.26172509011667	Time (s): 0.01543
[11] Improvement: 0.87820998422103	Time (s): 0.01529
[12] Improvement: 0.5205686544068158	Time (s): 0.01838
[13] Improvement: 0.2584411856587394	Time (s): 0.01523
[14] Improvement: 0.11901160411071032	Time (s): 0.01493
[15] Improvement: 0.053943474231346045	Time (s): 0.01513
[16] Improvement: 0.024498505976225715	Time (s): 0.01497
[17] Improvement: 0.011193219070264604	Time (s): 0.01479
[18] Improvement: 0.005145486393303145	Time (s): 0.0158

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (58/117)
2: (110/233)
7: (173/327)
6: (214/470)
8: (233/599)
0: (237/721)
9: (289/840)
5: (368/970)
3: (436/1074)
4: (513/1200)
Final recognition rate is 42.75 %
Evaluate on rest of FSDD set
0: (2/178)
3: (105/374)
9: (175/555)
7: (316/761)
4: (425/935)
5: (539/1105)
8: (559/1276)
1: (643/1459)
2: (736/1643)
6: (773/1800)
Final recognition rate is 42.94 %
Evaluate on wolfram test set
4: (217/2372)
8: (290/4724)
7: (1334/7101)
1: (1522/9471)
2: (1600/11844)
3: (2597/14200)
6: (3133/16569)
0: (3263/18945)
5: (3661/21302)
9: (4054/23666)
Final recognition rate is 17.13 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_5_1[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()

## **Mô hình 3**
- 3 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,3,1)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 73148.65842724007	Time (s): 0.01035
[2] Improvement: 596.8074574309867	Time (s): 0.01034
[3] Improvement: 61.379112052447454	Time (s): 0.01114
[4] Improvement: 11.587263191846432	Time (s): 0.01095
[5] Improvement: 3.432884597517841	Time (s): 0.01064
[6] Improvement: 2.956622795689327	Time (s): 0.01077
[7] Improvement: 2.968720608871081	Time (s): 0.01079
[8] Improvement: 0.9381774920402677	Time (s): 0.01113
[9] Improvement: 0.6183199905135552	Time (s): 0.01143
[10] Improvement: 3.1933957672372344	Time (s): 0.01057
[11] Improvement: 4.007758312058286	Time (s): 0.01081
[12] Improvement: 0.8195159413298825	Time (s): 0.01066
[13] Improvement: 0.40430884997476824	Time (s): 0.01056
[14] Improvement: 0.17636425182718085	Time (s): 0.01052
[15] Improvement: 0.07328172519919463	Time (s): 0.01043
[16] Improvement: 0.03523596110608196	Time (s): 0.01052
[17] Improvement: 0.01962478431960335	Time (s): 0.01254
[18] Improvement: 0.011935586968320422	Time (s): 0.

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (15/117)
2: (65/233)
7: (137/327)
6: (151/470)
8: (154/599)
0: (207/721)
9: (247/840)
5: (280/970)
3: (314/1074)
4: (377/1200)
Final recognition rate is 31.42 %
Evaluate on rest of FSDD set
0: (74/178)
3: (118/374)
9: (176/555)
7: (330/761)
4: (415/935)
5: (451/1105)
8: (465/1276)
1: (487/1459)
2: (570/1643)
6: (587/1800)
Final recognition rate is 32.61 %
Evaluate on wolfram test set
4: (542/2372)
8: (732/4724)
7: (1802/7101)
1: (1958/9471)
2: (2062/11844)
3: (2557/14200)
6: (2943/16569)
0: (3261/18945)
5: (3743/21302)
9: (4081/23666)
Final recognition rate is 17.24 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_3_1[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()